In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd   
from bs4 import BeautifulSoup   
import re 
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%time train_data = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
%time test_data = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3 )

In [ ]:
train_data.head()

In [ ]:
def review_to_words( raw_review ):
    source_text = BeautifulSoup(raw_review).get_text()      
    letters_only = re.sub("[^a-zA-Z]", " ", source_text) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    
    return(" ".join(meaningful_words))

In [ ]:
num_reviews = train_data["review"].size
clean_train_reviews = []
for i in range(0, num_reviews):
    if (i%1000 == 0):
        %time print(i/num_reviews * 100, '%')
    clean_train_reviews.append(review_to_words(train_data["review"][i] ) )

In [ ]:
num_reviews = test_data["review"].size
clean_test_reviews = [] 

for i in range(0,num_reviews):
    clean_review = review_to_words( test_data["review"][i] )
    clean_test_reviews.append( clean_review )

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 20000) 

%time train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()

In [ ]:
forest = RandomForestClassifier(n_estimators=100) 
%time forest = forest.fit( train_data_features, train_data["sentiment"] )

In [ ]:
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [ ]:
%time result = forest.predict(test_data_features)

In [ ]:
output = pd.DataFrame( data={"id":test_data["id"], "sentiment":result} )

output.to_csv( "submission.csv", index=False, quoting=3 )
print('Your submission is successfully saved!')